# Modelling Polymers using Weighted Directed Graphs

The data used in this tutorial is taken from ["A graph representation of molecular ensembles for polymer property prediction" (Chem. Sci. 2022,13, 10486-10498)](https://pubs.rsc.org/en/content/articlelanding/2022/SC/D2SC02839E). The original data file can be found [here](https://github.com/coleygroup/polymer-chemprop-data/blob/main/datasets/vipea/chemprop_inputs/dataset-poly_chemprop.csv).

## Introduction

From a chemical perspective the formation and use of polymers is relatively straightforward. However, their computational representation is challenging and often fails to capture all aspects of the polymer. One way to represent polymers involves using weighted directed graphs, which include stoichiometry, polymer types and the direction and weight of the bonds. This example notebook will go through the weighted directed graph representation of polymers and how they can be used to model polymers in Chemprop.

Alternative polymer representations rely on repetitive monomer representations to depict a complete polymer chain, which is both inefficient and unrealistic. Basic graph datasets can represent a single repeating unit of one or more monomers To account for the repeating patterns of different monomers at various open bonds, weights and directions can be assigned to different connection sites. Therefore, by incorporating weights and directions into a basic graph, we can efficiently manage the repeating architecture of polymers. 

The basic unit of a polymer is the monomer. Large numbers of repeating monomers form a polymer chain. Polymers can sometimes be composed of various types of monomers in different proportions, also known as co-polymers. We will illustrate this by using an example co-polymer and create a representation using a weighted directed graph structure.

Let's take two monomers given in SMILES notation `OB(O)c1cc(F)c(B(O)O)cc1F` and `Oc1cc(O)c(Br)c(O)c1Br`.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

In [ ]:
mA = "OB(O)c1cc(F)c(B(O)O)cc1F"
mB = "Oc1cc(O)c(Br)c(O)c1Br"
molA = Chem.MolFromSmiles(mA)
molB = Chem.MolFromSmiles(mB)

Draw.MolsToGridImage([molA, molB], legends=["Monomer A", "Monomer B"])

Both monomers exhibit functionalality attached to their benzene rings. Monomer A, contains two boronic acid groups. Monomer B, contains two bromine atoms. These two monomers can react together via. a Suzuki coupling reaction to form a polymeric species. By replacing these functionalities with open bonds we can connect the two monomers together to illustrate the resulting polymer. For the moment let's join two monomers at an open bond and represent the remaining open bonds with a virtual atom `[At]`.  

In [ ]:
mAB = "[At]c1cc(F)c(c2c(O)c([At])c(O)cc2(O))cc1(F)"
molAB = Chem.MolFromSmiles(mAB)

Draw.MolsToGridImage([molA, molB, molAB], legends=["Monomer A", "Monomer B", "Combined Molecule"])

For practical implementations however, this conversion process must be developed as the virtual atom `[At]` does not fully represent the structural type of the polymer or the connection pattern with other monomers.

## Polymeric Represenation

To convert the monomers into a single structure, we need to simulate the reaction process by eliminating the functional groups and marking the open bonds with indicies.

In [ ]:
def remove_duplicate_mols(mols):
    smiles = list(set([Chem.MolToSmiles(m, canonical=True) for m in mols]))
    return [Chem.MolFromSmiles(s) for s in smiles]

In [ ]:
molA_s = molA
for i in [1, 2]:
    smarts = f"[*:1]([B](-O)(-O))>>[*:1]-[*{i}]" # SMARTS to emulate the reaction of the boronic acid to an open bond
    reaction = AllChem.ReactionFromSmarts(smarts)
    products = reaction.RunReactants((molA_s,))
    products = remove_duplicate_mols([r[0] for r in products])
    molA_s = products[0]
    
Draw.MolsToGridImage([molA, molA_s], legends=["Original Monomer A", "Converted Monomer A"])

The boronic acid groups have now been replaced with an open bond and a corresponding index number. We can now repeat the process for monomer B.

In [ ]:
molB_s = molB
for i in [3, 4]:
    smarts = f"[*:1]Br>>[*:1]-[*{i}]" # SMARTS to emulate the reaction of the bromine to an open bond
    reaction = AllChem.ReactionFromSmarts(smarts)
    products = reaction.RunReactants((molB_s,))
    products = remove_duplicate_mols([r[0] for r in products])
    molB_s = products[0]
    
Draw.MolsToGridImage([molB, molB_s], legends=["Original Monomer B", "Converted Monomer B"])

To represent both monomers while keeping a valid SMILES, we can join both SMILES strings with a `.`.

In [ ]:
molA_smiles = Chem.MolToSmiles(molA_s, canonical=True)
molB_smiles = Chem.MolToSmiles(molB_s, canonical=True)
combined_AB_smiles = f"{molA_smiles}.{molB_smiles}"
combined_AB_mols = Chem.MolFromSmiles(combined_AB_smiles)
print(f"Combined AB SMILES: {combined_AB_smiles}")
Draw.MolsToGridImage([combined_AB_mols], legends=["Combined AB Mols"])

### Weight and Direction Assignments

In the previous step, we created a monomer wireframe capable of referencing different indexed open bonds to form various types of polymer chains. However, the goal is to condense the complete polymer chain architecture into a smaller yet robust representation using weights and directions. Weights can be assigned at two levels: to the individual monomer atoms and to each bond. While the direction is generally consistent across different monomers, there in an edge case where a bond refers to itself (also known as a self-loop or self-edge in graphs), where an open bond connects with the same open bond of another monomer with an identical structure. For example a bond between atom 1* of Monomer A and atom 2* of another Monomer A in the image above.

#### Weight assignments to monomer elements

Polymers can be composed of monomers contributing in varying proportions. The proportion that each monomer contributes is fairly straightforward. For example, if monomer A and monomer B contribute to the polymer in a 3:1 ratio, the weights assigned to monomer A and monomer B are 0.75 and 0.25 respectively. In this way, the contribution of each monomer fragment to the total polymer is represented by the weights of the nodes in the graph.

The stoichiometry of the polymer can impose limitations on the weights assigned to the monomers. For example, alternating polymers with the structure `ABABABAB...` must have an almost equal fraction of monomer A and monomer B. Whereas, in block and random co-polymers, the fractions can vary and have different float values.

For representational purposes we denote the weights of the monomers with a separator "|". For example for alternating polymers the monomer weights would be represented as `|0.5|0.5|`.

#### Weight assignments to bonds

Since we cannot replicate the complete repitition of the monomer, we must convert the stoichiometric constraints into edge weights in the node representation.

When determining a weight for the represenationation, we must divide the total contribution by the effective contribution. The total contribution value can be confusing since not all monomers are included in the chain. Therefore, we will use the same fragment ratio used for assigning weights to the monomer elements. For example, if the monomers are in a 3:1 ratio, we will consider monomer arrangments in multiples of 4. For block co-polymers, we would use "AAAABBBB".

##### 1. Weights for alternating polymers

For alternating polymers, the structure will resemble "ABABABAB". Therefore, monomers A and B will never connect to themselves, so the case of self-edges does not apply here. Considering the probability occurence, the bonds between A-B and B-A will each be assigned a weight of `0.5`.

In [ ]:
print(f"Combined AB SMILES: {combined_AB_smiles}")
Draw.MolsToGridImage([combined_AB_mols], legends=["Combined AB Mols"])

We represent the combination of atom indexes and bond weights for both the forward and reverse bond formation as a single polymer rule, separated by `:`. For example, if the weights of the bond formation from A to B is 0.25 and B to A is 0.75, it is represented as `1-2:0.25:0.75`. Multiple polymer rules are indicated starting with a `<` and separating each rule similarly. For instance, `<1-2:0.5:0.5<3-4:0.25:0.75` signifies connection between index 1 open bond and index 2 open bond with a weight of 0.5, and between index 3 open bond and index 4 open bond with weights 0.25 and 0.75 for the forward and reverse bond formation respectively. 

In our example case, there are 4 indexes open bonds, and self-edges are not present (i.e. 1-2 or 3-4 bonds cannot be formed). Therefore, the only possible bonds formed are 1-3, 1-4, 2-3 and 2-4. Each bond will have a weight of 0.5.

The polymer represenation of this is: `<1-3:0.5:0.5<1-4:0.5:0.5<2-3:0.5:0.5<2-4:0.5:0.5`

In [ ]:
# For an alternating co-polymer
alt_edge_content = [(1, 3, 0.5, 0.5),
                    (1, 4, 0.5, 0.5),
                    (2, 3, 0.5, 0.5),
                    (2, 4, 0.5, 0.5)]
alt_edge_string = "".join([f"<{e[0]}-{e[1]}:{e[2]}:{e[3]}" for e in alt_edge_content])
print(f"Polymer Rule String for alternate co-polymer: {alt_edge_string}")

##### 2. Weights for block co-polymers

Block copolymers are characterized by the structure 'AAAABBBB'. Where, monomer A occupies one region and monomer B occupies another.

To model the weight distribution of an 'AAAABBBB' block copolymer, we represent it using eight monomer units. A total of eight bonds can be formed within this sequence. Six of these bonds are homogeneous (A-A or B-B), resulting in a bond probability and weight of 6/8. Conversely, only one heterogeneous bond (A-B) is possible, yielding a probability and weight of 1/8. The bond weight distribution based on atom indices is as follows:

- 1-2: 6/8
- 3-4: 6/8
- 1-3: 1/8
- 1-4: 1/8
- 2-3: 1/8
- 2-4: 1/8

This information can be converted into a polymer rule:

In [ ]:
# For a block co-polymer with no self edges
block_edge_content_wo_self_edges = [(1, 2, 6/8, 6/8),  # within A
                                    (3, 4, 6/8, 6/8),  # within B
                                    (1, 3, 1/8, 1/8),  # between A and B
                                    (1, 4, 1/8, 1/8),
                                    (2, 3, 1/8, 1/8),
                                    (2, 4, 1/8, 1/8)]
block_no_self_edges_string = "".join([f"<{e[0]}-{e[1]}:{e[2]}:{e[3]}" for e in block_edge_content_wo_self_edges])
print(f"Polymer Rule String for Block without Self Edges: {block_no_self_edges_string}")

If we consider self-edges (i.e. the same monomer structure connects to itself at same open bond) the probabilty weight between homogeneous bonds will be altered. For example, the connection for A-A at index 1 can be at two open bonds i.e. 1-2 or 1-1 (self-edge) hence the previous probability will be halved resulting in 3/8. Rest of the heterogenous bond weights will remain the same. So resulting weights will be as follows.

- 1-2: 3/8
- 1-1: 3/8
- 2-2: 3/8
- 3-4: 3/8
- 3-3: 3/8
- 4-4: 3/8
- 1-3: 1/8
- 1-4: 1/8
- 2-3: 1/8
- 2-4: 1/8

This information can be converted into a polymer rule:

In [ ]:
# For a block co-polymer with self edges
block_edge_content = [(1, 2, 3/8, 3/8),  # within A
                      (1, 1, 3/8, 3/8),
                      (2, 2, 3/8, 3/8),
                      (3, 4, 3/8, 3/8),  # within B
                      (3, 3, 3/8, 3/8),
                      (4, 4, 1/8, 1/8),
                      (1, 3, 1/8, 1/8),  # between A and B
                      (1, 4, 1/8, 1/8),
                      (2, 3, 1/8, 1/8),
                      (2, 4, 1/8, 1/8)]
block_edges_string = "".join([f"<{e[0]}-{e[1]}:{e[2]}:{e[3]}" for e in block_edge_content])
print(f"Polymer Rule String for Block with Self Edges: {block_edges_string}")

##### 3. Weights for Random Polymers

The weight calculation for a random polymer chain is straightforward. Without self-edges, any indexed bond can connect to another open bond of the same monomer (e.g., 1-2 or 2-1) with a probability of 1/2. Similarly, any indexed bond connecting with an indexed open bond of another monomer type (e.g., 1-3, 1-4, 2-3, 2-4) has a probability of 1/3. When considering self-edges, every indexed bond can connect with every other indexed bond (i.e., at 4 possible bonding sites), resulting in a probability weight of 1/4. Thus, with or without self-edges, we can determine the following weight distribution.

With self-edge:
- 1-1 -> 0.25
- 1-2 -> 0.25
... (same for all bond connection)

Without self-edge:
- 1-2 -> 0.5
- 3-4 -> 0.5
- 1-3 -> 0.33
- 1-4 -> 0.33
- 2-3 -> 0.33
- 2-4 -> 0.33

In [ ]:
# For a random co-polymer with self edges
possible_bond_site_indexes = [1, 2, 3, 4]
random_edge_content = []
for i in possible_bond_site_indexes:
        for j in possible_bond_site_indexes:
                random_edge_content.append([i, j, 0.25, 0.25])
random_edges_string = "".join([f"<{e[0]}-{e[1]}:{e[2]}:{e[3]}" for e in random_edge_content])
print(f"Polymer Rule String for Block with Self Edges: {random_edges_string}")

In [ ]:
# For a random co-polymer without self edges
random_edge_content_wo_self_edge = [(1, 2, 1/2, 1/2),  # within A
                                    (3, 4, 1/2, 1/2),  # within B
                                    (1, 3, 1/3, 1/3),  # between A and B
                                    (1, 4, 1/3, 1/3),
                                    (2, 3, 1/3, 1/3),
                                    (2, 4, 1/3, 1/3)]
random_edges_wo_self_edge_string = "".join([f"<{e[0]}-{e[1]}:{e[2]:.2f}:{e[3]:.2f}" for e in random_edge_content_wo_self_edge])
print(f"Polymer Rule String for Random without Self Edges: {random_edges_wo_self_edge_string}")

In this manner, we can add weights of the monomer fragments and bonds to the graph data.

### Final Polymer Representation

In the previous steps, we created a wireframe for monomers with indexed open bonds and calculated the weights for bonds and elements based on stoichiometry and the monomer fragment distribution. Now, we need to combine these into a single string format to construct a weighted directed graph. We have the information in three parts:
 
1. The indexed monomer wireframe
2. Assigned weights to elements
3. Polymer rules containing polymer weights mapped with direction

We concatenate these three parts into a single string. In our case, we have a monomer indexed representation of two monomers given as `[1*]c1cc(F)c([2*])cc1F.[3*]c1c(O)cc(O)c([4*])c1O`. For the second part, the fractions of the two monomers were asssigned to be 0.5 for each, `|0.5|0.5|`.

Finally, in the polymer rules section, there could be three variations of the same monomer corresponding to different stoichiometries. For each monomer example considered, there can be at least three types (i.e., Alternating, Block, and Random). Each type will have a different polymer rules string concatenated, resulting in different string representations from the same monomer string.

*Note - There could be different cases, if there are self-edges possible in the monomer stoichiomentry.*

In [ ]:
# For our example 
monomer_fragment_string = "|0.5|0.5|"
poly_wireframe_w_weights = combined_AB_smiles + monomer_fragment_string
print(f"Repeatable weighted base representation: {poly_wireframe_w_weights}")

Once we have the base string representation, we will append the polymer rule strings to the end of the base string. Since the polymer rules for this polymer have already been determined in previous code blocks, we will use those strings to construct the final string.

#### i. Without self-edges

In [ ]:
polymer_alternate = poly_wireframe_w_weights + alt_edge_string
polymer_block = poly_wireframe_w_weights + block_no_self_edges_string
polymer_random = poly_wireframe_w_weights + random_edges_wo_self_edge_string
print(f"The string representation of alternating polymer: {polymer_alternate}")
print(f"The string representation of block polymer: {polymer_block}")
print(f"The string representation of random polymer: {polymer_random}")

#### ii. With self-edges

In case of self-edges, an alternating polymer is not possible. We can include the remaining cases of random and block polymer to form a string representation as follows.

In [ ]:
polymer_block_w_self_edge = poly_wireframe_w_weights + block_edges_string
polymer_random_w_self_edge = poly_wireframe_w_weights + random_edges_string 
print(f"The string representation of block polymer with self-edge: {polymer_block_w_self_edge}")
print(f"The string representation of random polymer wth self-edge: {polymer_random_w_self_edge}")

These data string representations are not widely adopted, as the use of weighted directed graphs in these fields is relatively recent and specifically compatible with Weighted Directed Message Passing Neural Networks (wD-MPNN). However, this string representation effectively stores all the necessary data to form a valid weighted directed graph structure from polymer data.

## Using Weighted Polymer Representations with Chemprop

Now that we have a representation of our polymer including the monomer and bond weightings we can use them to train a Chemprop model to predict the electron affinity of a set of polymers.

In [ ]:
import pandas as pd
from pathlib import Path

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

from chemprop import data, featurizers, models, nn

### Load in the dataset

We will use a reduced version of the dataset with only 100 datapoints to reduce training time. The full dataset can be found at the top of the notebook and will yield much better metrics.

In [ ]:
chemprop_dir = Path.cwd().parent
input_path = f"{chemprop_dir}/tests/data/regression/polymer/dataset-poly_chemprop_small.csv"

In [ ]:
num_workers = 0
smiles_column = "poly_chemprop_input"
target_columns = ["EA vs SHE (eV)"]

In [ ]:
df_input = pd.read_csv(input_path)
df_input

### Get Polymer representations and targets

In [ ]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

In [ ]:
smis[:5] # Show the first 5 polymer string representations

In [ ]:
ys[:5] # Show the first 5 targets

### Get polymer datapoints

In [ ]:
all_data = [data.PolymerDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

In [ ]:
all_data

### Perform data splitting for training, validation and testing

In [ ]:
# Available split types
list(data.SplitType.keys())

Split the data into a training, validation and test set.

In [ ]:
mols = [d.mol for d in all_data]
train_indicies, val_indicies, test_indicies = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data,
    train_indicies, val_indicies, test_indicies
)

### Get PolymerDataset

Create a dataset for the training, validation and test data using the `PolymerMolGraphFeaturizer()` and normalize the targets.

In [ ]:
featurizer = featurizers.PolymerMolGraphFeaturizer()

train_dataset = data.PolymerDataset(train_data[0], featurizer)
scaler = train_dataset.normalize_targets()

val_dataset = data.PolymerDataset(val_data[0], featurizer)
val_dataset.normalize_targets(scaler)

test_dataset = data.PolymerDataset(test_data[0], featurizer)

### Get DataLoader

Build dataloaders for the training, validation and test sets.

In [ ]:
train_loader = data.build_dataloader(train_dataset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dataset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dataset, num_workers=num_workers, shuffle=False)

## Change the Weighted Directed Message-Passing Neural Network (wD-MPNN) inputs here

To use weighting in the `nn.MPNN`, the `nn.wMPNN` method should be used inplace of the base `nn.MPNN`.

#### Message Passing

A `Message Passing` constructs molecule graphs using message passing to learn node-level representations. To ensure weighting is used for the bond message passing step we need to use `nn.WeightedBondMessagePassing`.

Options are `mp = nn.WeightedBondMessagePassing()` or `mp = nn.AtomMessagePassing()`.

In [ ]:
mp = nn.WeightedBondMessagePassing()

#### Aggregation

An `Aggregation` is responsible for constructing a graph-level representation from the set of node-level representations after message passing.

Available options can be found in `nn.agg.AggregationRegistry`, including:
- `agg = nn.MeanAggregation()`
- `agg = nn.SumAggregation()`
- `agg = nn.NormAggregation()`

In [ ]:
print(nn.agg.AggregationRegistry)

In [ ]:
agg = nn.MeanAggregation()

### Feed-Forward Network (FNN)

A `FFN` takes the aggregated representation and makes target predictions.

Available options can be found in `nn.PredictorRegistry`.

In [ ]:
print(nn.PredictorRegistry)

Create function to unscale the targets

In [ ]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

As we are performing a regression task, we will choose the `nn.RegressionFFN` FFN.

In [ ]:
ffn = nn.RegressionFFN(output_transform=output_transform)

### Batch Norm

A `BatchNorm` normalizes the outputs of the aggreation by re-centering and re-scaling.

In [ ]:
# Whether to use batch normalisation
batch_norm = True

### Metrics

`Metrics` are the ways to evaluate the performance of the model predictions.

Available options can be found in `metrics.MetricRegistry`.

In [ ]:
print(nn.metrics.MetricRegistry)

As we are performing a regression task we will select RMSE and MAE

In [ ]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] # Only the first metric is used for training and early stopping

### Construct the wD-MPNN

In [ ]:
wdmpnn = models.wMPNN(mp, agg, ffn, batch_norm, metric_list)
wdmpnn

## Set up the trainer

In [ ]:
# Configure model checkpointing
checkpointing = ModelCheckpoint(
    "checkpoints", # Directory where model checkpoints will be saved
    "best-{epoch}-{val_loss:.2f}", # Filename format for the checkpoints
    "val_loss", # Metric used to select the best checkpoint (based on validation loss)
    mode="min", # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True, # Always save the most recent checkpoint, even if its not the best
)

# Configure model early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', # Metric used to monitor early stopping
    mode='min',
    patience=15, # Number of epochs to wait for a lower metric value before early stopping
)

trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator="auto",
    devices="auto",
    max_epochs=20, # Number of epochs to train for
    callbacks=[checkpointing, early_stopping],
)

## Start training

In [ ]:
trainer.fit(wdmpnn, train_loader, val_loader)

## Test Results

In [ ]:
results = trainer.test(dataloaders=test_loader, ckpt_path="best")